In [1]:
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process 
import pandas as pd
import numpy as np
from collections import defaultdict
import re
import csv

In [98]:
stations_df = pd.read_csv('stations_zip.csv')
for column in stations_df.columns:    
    stations_df.rename(columns={column: column.lower().strip()}, inplace=True)

stations_df.drop(['latitude','longitude'], axis=1, inplace = True)

stations_df = stations_df.apply(lambda x: x.astype(str).str.lower())

stations_df.sort_values(by = ['name'], inplace = True)

In [99]:
stations_df.name = stations_df.name.str.replace('rd', '')
stations_df.name = stations_df.name.str.replace('th', '')
stations_df.line = stations_df.line.str.replace(' ', '')

stations_df.head()

,unnamed: 0,name,line,zipcode
0,0,103 st,abc,10025-4403
1,1,103 st,466,10029
2,2,103 st,1,10025
3,3,103 st - corona plaza,7,11368
4,4,104 st,as,11419


In [5]:
turnstiles_df = pd.read_csv('http://web.mta.info/developers/data/nyct/turnstile/turnstile_200606.txt')
for column in turnstiles_df.columns:    
    turnstiles_df.rename(columns={column: column.lower().strip()}, inplace=True)
    
turnstiles_df = turnstiles_df.apply(lambda x: x.astype(str).str.lower())

turnstiles_df.sort_values(by = ['station'], inplace = True)

In [7]:
turnstiles_df.linename = turnstiles_df.linename.str.replace(' ','')
turnstiles_df.station = turnstiles_df.station.str.replace(' ','')
turnstiles_df.head()

,c/a,unit,scp,station,linename,division,date,time,desc,entries,exits
33077,h008,r248,01-00-00,1av,l,bmt,06/01/2020,00:00:00,regular,303244,4993334
32653,h007a,r248,02-00-00,1av,l,bmt,05/31/2020,08:00:00,regular,93310,15662
32652,h007a,r248,02-00-00,1av,l,bmt,05/31/2020,04:00:00,regular,93308,15652
32651,h007a,r248,02-00-00,1av,l,bmt,05/31/2020,00:00:00,regular,93304,15652
32650,h007a,r248,02-00-00,1av,l,bmt,05/30/2020,20:00:00,regular,93293,15645


***Here's how we're solving it:***  
We create this dictionary, and then construct the same dictionary for the turnstile data.  
Then we run simple comparisons to match up by stop_name and line (the subway line), and then in the stations dictionary, append a field with the turnstyle ID associated with the station ID. Then we simply need to apply that dictionary and it's zipcodes to the turnstile dataframe. AND THEN IT'S LITERALLY DONE.  
We will have the turnstile data with zip-codes!

In [100]:
stations_dict = defaultdict(dict)

for i in stations_df.itertuples():
    x = re.sub('[^A-Za-z0-9]+', '', i[2]).lower()
    if ('ave' in x) and ('avenue' not in x):
        x = x.replace('ave','avenue')
    if ('av' in x) and ('ave' not in x) and ('avenue' not in x):
        x = x.replace("av","avenue")
    stations_dict[i[1]]['stop_name'] = x
    stations_dict[i[1]]['line'] = i[3]
    stations_dict[i[1]]['zipcode'] = i[4]
    stations_dict[i[1]]['turnstile_station_id'] = ''
    stations_dict[i[1]]['match_method'] = ''
    
turnstiles_dict = defaultdict(dict)

for i in turnstiles_df.itertuples():
    x = re.sub('[^A-Za-z0-9]+', '',i[4]).lower()
    if ('ave' in x) and ('avenue' not in x):
        x = x.replace('ave','avenue')
    if ('av' in x) and ('ave' not in x) and ('avenue' not in x):
        x = x.replace("av","avenue")
    turnstiles_dict[i[2]]['stop_name'] = x
    turnstiles_dict[i[2]]['line'] = i[5]

In [101]:
for i in stations_dict.keys():
            stations_dict[i]['turnstile_station_id'] = ''

def match_lines(i,j):
    print(j,i)
    match = False
    if len(j) <= len(i):
        if (j & i) == j:
            match = True
        if match:
            print('stats<=turns')
    elif len(j) >= len(i):
        if (i & j) == i:
            match = True
        if match:
            print('turns<=stats')

    return match

for j in stations_dict.keys():
    print('/\/\/\/\/\/\/\/\/\/\/')
    print('Trying to match ',stations_dict[j]['stop_name'])
    
    
    
    matched = False
    if not matched:
        print('Method 1')
        for i in turnstiles_dict.keys():
            
            if turnstiles_dict[i]['stop_name'] == stations_dict[j]['stop_name']:
                print('With ',turnstiles_dict[i]['stop_name'])
                print('Name match1')
                if match_lines(set(stations_dict[j]['line']),set(turnstiles_dict[i]['line'])):
                    print('Succes!1')
                    stations_dict[j]['turnstile_station_id'] = i 
                    stations_dict[j]['match_method'] = 1
                    matched = True
                    break
    if not matched:
        print('Method 2')
        for i in turnstiles_dict.keys():
            
            if stations_dict[j]['stop_name'] in turnstiles_dict[i]['stop_name']:
                print('With ',turnstiles_dict[i]['stop_name'])
                print('Name match2')
                if match_lines(set(stations_dict[j]['line']),set(turnstiles_dict[i]['line'])):
                    print('Succes!2')
                    stations_dict[j]['turnstile_station_id'] = i 
                    stations_dict[j]['match_method'] = 2
                    matched = True
                    break
    if not matched: 
        print('Method 3')
        for i in turnstiles_dict.keys():
            
            if turnstiles_dict[i]['stop_name'] in stations_dict[j]['stop_name']:
                print('With ',turnstiles_dict[i]['stop_name'])
                print('Name match3')
                if match_lines(set(stations_dict[j]['line']),set(turnstiles_dict[i]['line'])):
                    print('Succes!3')
                    stations_dict[j]['turnstile_station_id'] = i 
                    stations_dict[j]['match_method'] = 3
                    matched = True
                    break
    if not matched:
        print('Method 4')
        for i in turnstiles_dict.keys():
            if fuzz.ratio(turnstiles_dict[i]['stop_name'], stations_dict[j]['stop_name']) > 70:
                print('With ',turnstiles_dict[i]['stop_name'])
                print('Name match4') 
                if match_lines(set(stations_dict[j]['line']),set(turnstiles_dict[i]['line'])):
                    print('Success!4')
                    stations_dict[j]['turnstile_station_id'] = i 
                    stations_dict[j]['match_method'] = 4
                    break
    else:
        continue

        

/\/\/\/\/\/\/\/\/\/\/
Trying to match  103st
Method 1
With  103st
Name match1
{'6'} {'c', 'a', 'b'}
With  103st
Name match1
{'c', 'b'} {'c', 'a', 'b'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  103st
Method 1
With  103st
Name match1
{'6'} {'6', '4'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  103st
Method 1
With  103st
Name match1
{'6'} {'1'}
With  103st
Name match1
{'c', 'b'} {'1'}
With  103st
Name match1
{'1'} {'1'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  103stcoronaplaza
Method 1
Method 2
Method 3
With  103st
Name match3
{'6'} {'7'}
With  103st
Name match3
{'c', 'b'} {'7'}
With  103st
Name match3
{'1'} {'7'}
With  103stcorona
Name match3
{'7'} {'7'}
stats<=turns
Succes!3
/\/\/\/\/\/\/\/\/\/\/
Trying to match  104st
Method 1
With  104st
Name match1
{'j', 'z'} {'s', 'a'}
With  104st
Name match1
{'a'} {'s', 'a'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  104102ndsts
Method 1
Method 2
Method 3
Method 4
/\/\/\/\/\/

{'r'} {'d', 'r', 'n'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  28st
Method 1
With  28st
Name match1
{'6'} {'6', '4'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  28st
Method 1
With  28st
Name match1
{'6'} {'2'}
With  28st
Name match1
{'1'} {'2'}
With  28st
Name match1
{'w', 'r', 'n'} {'2'}
Method 2
With  28st
Name match2
{'6'} {'2'}
With  28st
Name match2
{'1'} {'2'}
With  28st
Name match2
{'w', 'r', 'n'} {'2'}
Method 3
With  28st
Name match3
{'6'} {'2'}
With  28st
Name match3
{'1'} {'2'}
With  28st
Name match3
{'w', 'r', 'n'} {'2'}
Method 4
With  18st
Name match4
{'1'} {'2'}
With  21st
Name match4
{'g'} {'2'}
With  23st
Name match4
{'e', 'c'} {'2'}
With  23st
Name match4
{'m', 'f'} {'2'}
With  23st
Name match4
{'m', 'f'} {'2'}
With  23st
Name match4
{'w', 'r', 'n'} {'2'}
With  23st
Name match4
{'1'} {'2'}
With  23st
Name match4
{'6'} {'2'}
With  238st
Name match4
{'1'} {'2'}
With  25st
Name match4
{'r'} {'2'}
With  25st
Name match4
{'r'} {'2'}
With 

/\/\/\/\/\/\/\/\/\/\/
Trying to match  75avenue
Method 1
With  75avenue
Name match1
{'e', 'f'} {'e', 'f'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  75steldertln
Method 1
Method 2
Method 3
Method 4
With  75stelderts
Name match4
{'j', 'z'} {'j', 'z'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  77st
Method 1
With  77st
Name match1
{'6'} {'r'}
With  77st
Name match1
{'r'} {'r'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  77st
Method 1
With  77st
Name match1
{'6'} {'6', '4'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  79st
Method 1
With  79st
Name match1
{'d'} {'2'}
With  79st
Name match1
{'1'} {'2'}
Method 2
With  79st
Name match2
{'d'} {'2'}
With  79st
Name match2
{'1'} {'2'}
With  jamaica179st
Name match2
{'f'} {'2'}
Method 3
With  79st
Name match3
{'d'} {'2'}
With  79st
Name match3
{'1'} {'2'}
Method 4
With  49st
Name match4
{'w', 'q', 'r', 'n'} {'2'}
With  57st
Name match4
{'f'} {'2'}
With  59st
Name match4
{'6', 'q

Name match4
{'n'} {'d'}
With  baypkwy
Name match4
{'d'} {'d'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  baypky
Method 1
Method 2
Method 3
Method 4
With  baypkwy
Name match4
{'f'} {'f'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  bayridge95st
Method 1
With  bayridge95st
Name match1
{'r'} {'r'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  bayridgeavenue
Method 1
With  bayridgeavenue
Name match1
{'r'} {'r'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  baychesteravenue
Method 1
With  baychesteravenue
Name match1
{'5'} {'5'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  beach105st
Method 1
With  beach105st
Name match1
{'s', 'a'} {'s', 'a'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  beach25st
Method 1
With  beach25st
Name match1
{'a'} {'a'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  beach36st
Method 1
With  beach36st
Name match1
{'a'} {'a'}
stats<=turns
Succes!1
/\/\/\/\/\

Name match4
{'c', 'b'} {'1'}
With  cathedralpkwy
Name match4
{'1'} {'1'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  centralavenue
Method 1
With  centralavenue
Name match1
{'m'} {'m'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  centralparknor110st
Method 1
Method 2
Method 3
With  110st
Name match3
{'6'} {'3'}
Method 4
With  centralpkn110
Name match4
{'2', '3'} {'3'}
turns<=stats
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  chambersst
Method 1
With  chambersst
Name match1
{'j', '6', 'z', '5', '4'} {'j', 'z'}
turns<=stats
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  chambersst
Method 1
With  chambersst
Name match1
{'j', '6', 'z', '5', '4'} {'2', '3', '1'}
With  chambersst
Name match1
{'2', '3', '1'} {'2', '3', '1'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  chambersst
Method 1
With  chambersst
Name match1
{'j', '6', 'z', '5', '4'} {'c', 'a'}
With  chambersst
Name match1
{'2', '3', '1'} {'c', 'a'}
Method 2
With  chambersst
Name m

/\/\/\/\/\/\/\/\/\/\/
Trying to match  fultonst
Method 1
With  fultonst
Name match1
{'j', '2', 'c', 'z', '5', '3', 'a', '4'} {'5'}
turns<=stats
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  fultonst
Method 1
With  fultonst
Name match1
{'j', '2', 'c', 'z', '5', '3', 'a', '4'} {'3'}
turns<=stats
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  fultonst
Method 1
With  fultonst
Name match1
{'j', '2', 'c', 'z', '5', '3', 'a', '4'} {'j', 'z'}
turns<=stats
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  gatesavenue
Method 1
With  gatesavenue
Name match1
{'j', 'z'} {'j', 'z'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  grahamavenue
Method 1
With  grahamavenue
Name match1
{'l'} {'l'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  grandarmyplaza
Method 1
Method 2
Method 3
With  grandarmyplaz
Name match3
{'2', '3'} {'2', '3', '4'}
stats<=turns
Succes!3
/\/\/\/\/\/\/\/\/\/\/
Trying to match  grandavenuenewtown
Method 1
Method 2
Method 3
With  avenuen
Name match3
{'f

stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  morganavenue
Method 1
With  morganavenue
Name match1
{'l'} {'l'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  morrispark
Method 1
With  morrispark
Name match1
{'5'} {'5'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  morrisonavenuesoundview
Method 1
Method 2
Method 3
Method 4
With  morisnavenuesndvw
Name match4
{'6'} {'6'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  mosholupkwy
Method 1
With  mosholupkwy
Name match1
{'4'} {'4'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  mtedenavenue
Method 1
With  mtedenavenue
Name match1
{'4'} {'4'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  myrtleavenue
Method 1
With  myrtleavenue
Name match1
{'j', 'm', 'z'} {'j', 'm', 'z'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  myrtleavenue
Method 1
With  myrtleavenue
Name match1
{'j', 'm', 'z'} {'l'}
Method 2
With  myrtleavenue
Name match2
{'j', 'm',

Trying to match  sutphinblvdarcheravenue
Method 1
Method 2
Method 3
With  sutphinblvd
Name match3
{'f'} {'j', 'z', 'e'}
Method 4
With  sutphinarcher
Name match4
{'j', 'z', 'e'} {'j', 'z', 'e'}
stats<=turns
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  sutteravenue
Method 1
With  sutteravenue
Name match1
{'l'} {'l'}
stats<=turns
Succes!1
/\/\/\/\/\/\/\/\/\/\/
Trying to match  sutteravenuerutlandroad
Method 1
Method 2
Method 3
With  sutteravenue
Name match3
{'l'} {'4'}
Method 4
With  sutteravenuerutld
Name match4
{'3'} {'4'}
/\/\/\/\/\/\/\/\/\/\/
Trying to match  timessq42ndst
Method 1
Method 2
Method 3
Method 4
With  timessq42st
Name match4
{'2', 'q', 'n', '7', 'c', 'r', 's', '1', '3', 'e', 'w', 'a'} {'2', '3', '1'}
turns<=stats
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  timessq42ndst
Method 1
Method 2
Method 3
Method 4
With  timessq42st
Name match4
{'2', 'q', 'n', '7', 'c', 'r', 's', '1', '3', 'e', 'w', 'a'} {'7'}
turns<=stats
Success!4
/\/\/\/\/\/\/\/\/\/\/
Trying to match  t

In [102]:
count = 0
o_count = 0
for k in stations_dict.keys():
    if stations_dict[k]['turnstile_station_id'] != '':
        count+=1
    elif stations_dict[k]['turnstile_station_id'] == '':
        o_count+=1

print(count)
print(o_count)

410
60


In [97]:
turnstiles_dict

defaultdict(dict,
            {'r248': {'stop_name': '1avenue', 'line': 'l'},
             'r180': {'stop_name': '103st', 'line': '6'},
             'r314': {'stop_name': '103st', 'line': 'bc'},
             'r191': {'stop_name': '103st', 'line': '1'},
             'r208': {'stop_name': '103stcorona', 'line': '7'},
             'r007': {'stop_name': '104st', 'line': 'jz'},
             'r354': {'stop_name': '104st', 'line': 'a'},
             'r181': {'stop_name': '110st', 'line': '6'},
             'r355': {'stop_name': '111st', 'line': 'a'},
             'r008': {'stop_name': '111st', 'line': 'j'},
             'r310': {'stop_name': '111st', 'line': '7'},
             'r182': {'stop_name': '116st', 'line': '6'},
             'r324': {'stop_name': '116st', 'line': '23'},
             'r333': {'stop_name': '116st', 'line': 'bc'},
             'r159': {'stop_name': '116stcolumbia', 'line': '1'},
             'r009': {'stop_name': '121st', 'line': 'jz'},
             'r132': {'stop_name'

In [103]:
csv_columns = ['stop_name','line','zipcode','turnstile_station_id','match_method']

In [104]:
with open('test.csv', 'w') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
    writer.writeheader()
    for key, value in stations_dict.items():
        writer.writerow(value)

In [95]:
new_stations_df = pd.read_csv('test.csv')

In [14]:
new_stations_df.head()

,stop_name,line,zipcode,turnstile_station_id,match_method
0,103st,abc,10025-4403,r314,1.0
1,103st,466,10029,r180,1.0
2,103st,1,10025,r191,1.0
3,103stcoronaplaza,7,11368,r208,3.0
4,104st,as,11419,r354,1.0


In [15]:
new_stations_df.zipcode.value_counts()

11207     23
11101     14
11223     13
10003     11
11201     11
          ..
10044      1
12692      1
100014     1
10028      1
11102      1
Name: zipcode, Length: 130, dtype: int64